In [134]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils

In [2]:
df = pd.read_csv('final file hopefully no errors.csv')
df['Len'] = [len(s) for s in df['Seq'].to_list()]

dfp = df.loc[df['Gen'] == 'ssRNA(+)'].reset_index(drop = True)
dfn = df.loc[df['Gen'] == 'ssRNA(-)'].reset_index(drop = True)

res = pd.read_csv('res 340.csv')
resp = res.iloc[:2862, :].reset_index(drop = True)
resn = res.iloc[2862:, :].reset_index(drop = True)

In [16]:
rpd = pd.concat([resp.quantile(q=0.05), resp.quantile(q=0.10), resp.quantile(q=0.90), resp.quantile(q=0.95)], axis=1)
rnd = pd.concat([resn.quantile(q=0.05), resn.quantile(q=0.10), resn.quantile(q=0.90), resn.quantile(q=0.95)], axis=1)

In [30]:
odf = pd.DataFrame(index = rpd.index, columns= rpd.columns)

for i in rpd.index:
    odf.loc[i,0.05] = max(rpd.loc[i,0.05], rnd.loc[i,0.05])
    odf.loc[i,0.10] = max(rpd.loc[i,0.10], rnd.loc[i,0.10])
    odf.loc[i,0.90] = min(rpd.loc[i,0.90], rnd.loc[i,0.90])
    odf.loc[i,0.95] = min(rpd.loc[i,0.95], rnd.loc[i,0.95])

In [35]:
odf['in 90'] = 0
odf['in 80'] = 0

for i in res.index:
    for c in res.columns:
        if odf.loc[c,0.05] < res.loc[i,c] < odf.loc[c,0.95]:
            odf.loc[c, 'in 90'] = odf.loc[c, 'in 90'] + 1
            
            if odf.loc[c,0.10] < res.loc[i,c] < odf.loc[c,0.90]:
                odf.loc[c, 'in 80'] = odf.loc[c, 'in 80'] + 1

In [60]:
odf.sort_values(by=['in 80', 'in 90'])[:5]

,0.05,0.1,0.9,0.95,in 90,in 80
ATC,0.0341546,0.0372251,0.0341369,0.0363116,150,0
ATCA,0.011796,0.0126798,0.0121456,0.0131263,242,0
ATCT,0.00917614,0.00995548,0.00977367,0.010607,350,0
TATC,0.00725136,0.00851144,0.0085953,0.00958841,845,16
TGGT,0.00692177,0.00852914,0.0088329,0.00927491,508,57


In [ ]:
Challenger

In [311]:
def split(df1):
    df1.loc[pndf['Gen'] == 'ssRNA(+)', 'Gen'] = 0
    df1.loc[pndf['Gen'] == 'ssRNA(-)', 'Gen'] = 1
    p = df1.loc[:2861].sample(frac=1).reset_index(drop = True)
    n = df1.loc[2862:].sample(frac=1).reset_index(drop = True)
    ps1 = p.iloc[:1144]
    ns1 = n.iloc[:112]
    ps2 = p.iloc[1144:2288].reset_index(drop = True)
    ns2 = n.iloc[112:224].reset_index(drop = True)
    ps3 = p.iloc[2288:].reset_index(drop = True)
    ns3 = n.iloc[224:].reset_index(drop = True)
    
    return (ps1,ns1,ps2,ns2,ps3,ns3)

In [312]:
def featsrc(resp, resn):    
    
    res = pd.concat([resp,resn], axis = 0).reset_index(drop = True)
    
    rpd = pd.concat([resp.quantile(q=0.05), resp.quantile(q=0.10), resp.quantile(q=0.90), resp.quantile(q=0.95)], axis=1)
    rnd = pd.concat([resn.quantile(q=0.05), resn.quantile(q=0.10), resn.quantile(q=0.90), resn.quantile(q=0.95)], axis=1)
    
    odf = pd.DataFrame(index = rpd.index, columns= rpd.columns)

    for i in rpd.index:
        odf.loc[i,0.05] = max(rpd.loc[i,0.05], rnd.loc[i,0.05])
        odf.loc[i,0.10] = max(rpd.loc[i,0.10], rnd.loc[i,0.10])
        odf.loc[i,0.90] = min(rpd.loc[i,0.90], rnd.loc[i,0.90])
        odf.loc[i,0.95] = min(rpd.loc[i,0.95], rnd.loc[i,0.95])
    
    odf['in 90'] = 0
    odf['in 80'] = 0

    for i in res.index:
        for c in res.columns:
            if odf.loc[c,0.05] < res.loc[i,c] < odf.loc[c,0.95]:
                odf.loc[c, 'in 90'] = odf.loc[c, 'in 90'] + 1

                if odf.loc[c,0.10] < res.loc[i,c] < odf.loc[c,0.90]:
                    odf.loc[c, 'in 80'] = odf.loc[c, 'in 80'] + 1
                    
    return odf.sort_values(by=['in 80', 'in 90'])[:5].index

In [313]:
def rfimp(feats,pnrtr,pnrts):
    
    r1 = pnrtr[feats].copy()
    r2 = pnrts[feats].copy()
    pnrtr = pnrtr.iloc[:,:8].drop(columns=['Id', 'MolTyp', 'Topol', 'Des', 'Len'])
    pnrts = pnrts.iloc[:,:8].drop(columns=['Id', 'MolTyp', 'Topol', 'Des', 'Len'])
    
    pnrtr = pd.concat([pnrtr, r1], axis = 1)
    pnrts = pd.concat([pnrts, r2], axis = 1)
         
    #Splitting features and labels
    X_train = pnrtr.iloc[:, 3:].values.astype(float)
    X_test = pnrts.iloc[:, 3:].values.astype(float)
    y_train = pnrtr.iloc[:, 1].values.astype(float)
    y_test = pnrts.iloc[:, 1].values.astype(float)
       
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    return cm

In [ ]:
pndf = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')
df1 = pd.concat([pndf, res], axis = 1)

aclist = []

for i in range(20):
    a,b,c,d,e,f = split(df1)
    ps1 = a.iloc[:,:8]
    ns1 = b.iloc[:,:8]
    rp1 = a.iloc[:,8:]
    rn1 = b.iloc[:,8:]
    pnrtr = pd.concat([c,d], axis = 0)
    pnrts = pd.concat([e,f], axis = 0)    
    
    feats = featsrc(rp1,rn1)
    cm = rfimp(feats, pnrtr, pnrts)

    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))

print(np.mean(aclist))

0.9715639810426541
0.9699842022116903
0.9794628751974723
0.9826224328593997
0.9652448657187994
0.966824644549763
0.981042654028436
0.9794628751974723
0.9715639810426541
0.9715639810426541
0.9699842022116903
0.9684044233807267
0.9715639810426541


Negative control

In [ ]:
pndf = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')
df1 = pd.concat([pndf, res], axis = 1)

for i1 in range(10):
    df1['Gen'] = 0
    rindex = np.random.choice(len(df1), 283, replace=False)
    for i in rindex:
        df1.loc[i,'Gen', 'Gen'] = 1

    aclist = []

    for i2 in range(20):
        a,b,c,d,e,f = split(df1)
        ps1 = a.iloc[:,:8]
        ns1 = b.iloc[:,:8]
        rp1 = a.iloc[:,8:]
        rn1 = b.iloc[:,8:]
        pnrtr = pd.concat([c,d], axis = 0)
        pnrts = pd.concat([e,f], axis = 0)    

        feats = featsrc(rp1,rn1)
        cm = rfimp(feats, pnrtr, pnrts)

        aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
        print((cm[0][0] + cm[1][1]) / (cm.sum()))

    print(np.mean(aclist))

In [127]:
def featsrc():    
    pndf['Gen'] = 0
    rindex = np.random.choice(len(res), 283, replace=False)
    
    for i in rindex:
        pndf.loc[i,'Gen'] = 1
    
    pind = list(pndf.loc[pndf['Gen'] == 0].index)
    nind = list(pndf.loc[pndf['Gen'] == 1].index)
        
    resp = res.loc[pind, :]
    resn = res.loc[nind, :]
    
    rpd = pd.concat([resp.quantile(q=0.05), resp.quantile(q=0.10), resp.quantile(q=0.90), resp.quantile(q=0.95)], axis=1)
    rnd = pd.concat([resn.quantile(q=0.05), resn.quantile(q=0.10), resn.quantile(q=0.90), resn.quantile(q=0.95)], axis=1)
    
    odf = pd.DataFrame(index = rpd.index, columns= rpd.columns)

    for i in rpd.index:
        odf.loc[i,0.05] = max(rpd.loc[i,0.05], rnd.loc[i,0.05])
        odf.loc[i,0.10] = max(rpd.loc[i,0.10], rnd.loc[i,0.10])
        odf.loc[i,0.90] = min(rpd.loc[i,0.90], rnd.loc[i,0.90])
        odf.loc[i,0.95] = min(rpd.loc[i,0.95], rnd.loc[i,0.95])
    
    odf['in 90'] = 0
    odf['in 80'] = 0

    for i in res.index:
        for c in res.columns:
            if odf.loc[c,0.05] < res.loc[i,c] < odf.loc[c,0.95]:
                odf.loc[c, 'in 90'] = odf.loc[c, 'in 90'] + 1

                if odf.loc[c,0.10] < res.loc[i,c] < odf.loc[c,0.90]:
                    odf.loc[c, 'in 80'] = odf.loc[c, 'in 80'] + 1
                    
    return (pndf, odf.sort_values(by=['in 80', 'in 90'])[:5].index)

In [132]:
def rfimp(pn, feats):
    res = pd.read_csv('res 340.csv')
    r = res[feats].copy()
    pnr = pd.concat([pn, r], axis = 1)
    pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des', 'Len'])
    
    ps = pnr.loc[pnr['Gen'] == 0]
    ns = pnr.loc[pnr['Gen'] == 1]

    pnrs = pd.concat([ps,ns])
         
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    return cm
    

In [137]:
aclist = []

for i in range(20):
    pn, feats = featsrc()
    cm = rfimp(pn, feats)

    aclist.append((cm[0][0] + cm[1][1]) / (cm.sum()))
    print((cm[0][0] + cm[1][1]) / (cm.sum()))

print(np.mean(aclist))

0.9072426937738246
0.8970775095298602


KeyboardInterrupt: 